# 初めに
ここで書くのは、最初のコード。
あはは

In [1]:
print ("Hello, world")

Hello, world


In [1]:
print ("Hello, World!")

Hello, World!


# Hello

In [4]:
This is a pen.

SyntaxError: invalid syntax (<ipython-input-4-d4d46e90cd7b>, line 1)